In [35]:
import pickle as pkl
import pandas as pd
from tqdm.notebook import tqdm

import create_history
import datetime

# Creating Listing Histories for CRMLS

In [53]:
for i in tqdm(range(12)):
    with open(f'data/pw_df_{i}.pkl', 'rb') as f:
        pw_df = pkl.load(f)
    create_history.create_listings(pw_df)[['id', 'standard_status','events']].to_csv(f'data/out/crmls_backfill_{i}.csv')


  0%|          | 0/12 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Transforming Listing Histories

In [24]:
with open(f'data/pw_df_{0}.pkl', 'rb') as f:
        pw_df = pkl.load(f)
        pw_df = create_history.create_listings(pw_df)
pw_df.head()

,id,listing_id,listing_key,close_date,contingent_date,expiration_date,listing_contract_date,purchase_contract_date,withdrawn_date,list_price,original_list_price,close_price,price_change_timestamp,standard_status,previous_list_price,status_change_timestamp,coming_soon_timestamp,events
0,531108,P482283,4304827,2006-01-27,None,2006-02-13,2005-11-18,2006-01-09,None,655000.00,699900.00,655000.00,2023-11-21 16:16:24.512301,Closed,None,2023-11-21 16:16:24.508916,None,"[(100, 2005-11-18, 699900.00), (240, 2006-01-0..."
1,531111,11518657,5217920,2011-05-11,None,2011-07-04,2011-04-05,2011-05-11,2011-05-04,3250.00,3250.00,3500.00,2023-11-21 16:16:24.802937,Closed,None,2023-11-21 16:16:24.800147,None,"[(100, 2011-04-05, 3250.00), (240, 2011-05-11,..."
2,531114,OC21232979,367508705,2021-12-01,None,2021-12-31,2021-10-21,2021-11-09,None,635000.00,635000.00,660000.00,2023-11-21 16:16:25.488849,Closed,None,2023-11-21 16:16:25.486274,None,"[(100, 2021-10-21, 635000.00), (240, 2021-11-0..."
3,531118,TR14204495,7868871,2014-10-04,None,2014-12-31,2014-09-22,2014-10-04,None,2750.00,2750.00,2700.00,2023-11-21 16:16:26.696619,Closed,None,2023-11-21 16:16:26.694795,None,"[(100, 2014-09-22, 2750.00), (240, 2014-10-04,..."
4,531120,00087285,10773243,2000-05-17,None,2000-09-13,2000-03-15,2000-03-23,None,475000.00,475000.00,475000.00,2023-11-21 16:16:26.986836,Closed,None,2023-11-21 16:16:26.984016,None,"[(100, 2000-03-15, 475000.00), (240, 2000-03-2..."


In [46]:
str(pw_df['events'][42][0][1]) + ' 00:00:00'

'2014-11-18 00:00:00'

In [51]:
def transform_histories(events):
    status = []
    d = []
    price = []
    for event in events:
        status.append(str(event[0]))
        if type(event[1]) == datetime.date:
            d.append(str(event[1]) + ' 00:00:00')
        else:
            d.append(str(event[1]))
        price.append(str(event[2]))
    return status, d, price

In [47]:
transform_histories(pw_df['events'][42])

(['100', '620'],
 ['2014-11-18 00:00:00', '2023-11-21 16:16:59.101017'],
 ['199500.00', '199500.00'])

In [55]:
for j in tqdm(range(12)):
    lh = {'id': [], 'status_code': [], 'price': [], 'time': []}
    with open(f'data/pw_df_{j}.pkl', 'rb') as f:
        pw_df = pkl.load(f)
    tmp = create_history.create_listings(pw_df)[['id', 'standard_status','events']]
    for i, e in zip(tmp['id'], tmp['events']):
        s, d, p = transform_histories(e)
        lh['status_code'] += s
        lh['price'] += p
        lh['time'] += d
        lh['id'] += [i] * len(s)
    
    pd.DataFrame(lh).to_csv(f'data/crmls_lh_backfill/crmls_backfill_{j}.csv', index=False)

  0%|          | 0/12 [00:00<?, ?it/s]